# Self-Discover 에이전트

[Self-Discover 논문](https://arxiv.org/pdf/2402.03620.pdf)의 구현입니다.

[@catid의 구현](https://github.com/catid/self-discover/tree/main?tab=readme-ov-file)을 기반으로 합니다.


## 설정

먼저 필요한 패키지를 설치하고 API 키를 설정하겠습니다.

In [ ]:
%%capture --no-stderr
%pip install -U --quiet langchain langgraph langchain_openai

In [ ]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


_set_if_undefined("OPENAI_API_KEY")

<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요. 
    </p>
</div>   

## 프롬프트 정의

In [1]:
from langchain import hub

select_prompt = hub.pull("hwchase17/self-discovery-select")
print("Self-Discovery Select Prompt:")
select_prompt.pretty_print()
print("Self-Discovery Select Response:")
adapt_prompt = hub.pull("hwchase17/self-discovery-adapt")
adapt_prompt.pretty_print()
structured_prompt = hub.pull("hwchase17/self-discovery-structure")
print("Self-Discovery Structured Prompt:")
structured_prompt.pretty_print()
reasoning_prompt = hub.pull("hwchase17/self-discovery-reasoning")
print("Self-Discovery Structured Response:")
reasoning_prompt.pretty_print()

Self-Discovery Select Prompt:
Select several reasoning modules that are crucial to utilize in order to solve the given task:

All reasoning module descriptions:
{reasoning_modules}

Task: {task_description}

Select several modules are crucial for solving the task above:

Self-Discovery Select Response:
Rephrase and specify each reasoning module so that it better helps solving the task:

SELECTED module descriptions:
{selected_modules}

Task: {task_description}

Adapt each reasoning module description to better solve the task:

Self-Discovery Structured Prompt:
Operationalize the reasoning modules into a step-by-step reasoning plan in JSON format:

Here's an example:

Example task:

If you follow these instructions, do you return to the starting point? Always face forward. Take 1 step backward. Take 9 steps left. Take 2 steps backward. Take 6 steps forward. Take 4 steps forward. Take 4 steps backward. Take 3 steps right.

Example reasoning structure:

{
    "Position after instruction 1

## 그래프 정의

In [2]:
from typing import Optional
from typing_extensions import TypedDict

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from langgraph.graph import END, START, StateGraph


class SelfDiscoverState(TypedDict):
    reasoning_modules: str
    task_description: str
    selected_modules: Optional[str]
    adapted_modules: Optional[str]
    reasoning_structure: Optional[str]
    answer: Optional[str]


model = ChatOpenAI(temperature=0, model="gpt-4-turbo-preview")


def select(inputs):
    select_chain = select_prompt | model | StrOutputParser()
    return {"selected_modules": select_chain.invoke(inputs)}


def adapt(inputs):
    adapt_chain = adapt_prompt | model | StrOutputParser()
    return {"adapted_modules": adapt_chain.invoke(inputs)}


def structure(inputs):
    structure_chain = structured_prompt | model | StrOutputParser()
    return {"reasoning_structure": structure_chain.invoke(inputs)}


def reason(inputs):
    reasoning_chain = reasoning_prompt | model | StrOutputParser()
    return {"answer": reasoning_chain.invoke(inputs)}


graph = StateGraph(SelfDiscoverState)
graph.add_node(select)
graph.add_node(adapt)
graph.add_node(structure)
graph.add_node(reason)
graph.add_edge(START, "select")
graph.add_edge("select", "adapt")
graph.add_edge("adapt", "structure")
graph.add_edge("structure", "reason")
graph.add_edge("reason", END)
app = graph.compile()

## 그래프 실행

In [ ]:
reasoning_modules = [
    "1. 그 문제를 해결하는 데 도움이 되는 실험을 어떻게 고안할 수 있을까요?",
    "2. 이 문제를 해결하기 위한 아이디어 목록을 작성하고 진전이 있는지 확인하기 위해 문제에 하나씩 적용하세요.",
    # "3. 이 문제의 진행 상황을 어떻게 측정할 수 있을까요?",
    "4. 문제를 더 쉽게 해결할 수 있도록 어떻게 단순화할 수 있을까요?",
    "5. 이 문제의 기본 가정은 무엇입니까?",
    "6. 각 솔루션의 잠재적인 위험과 단점은 무엇입니까?",
    "7. 이 문제에 대한 대안적인 관점이나 시각은 무엇입니까?",
    "8. 이 문제와 그 솔루션의 장기적인 영향은 무엇입니까?",
    "9. 이 문제를 더 작고 관리 가능한 부분으로 어떻게 나눌 수 있을까요?",
    "10. 비판적 사고: 이 스타일은 다양한 관점에서 문제를 분석하고, 가정에 의문을 제기하고, 사용 가능한 증거나 정보를 평가하는 것을 포함합니다. 논리적 추론, 증거 기반 의사 결정 및 사고의 잠재적 편향이나 결함을 식별하는 데 중점을 둡니다.",
    "11. 창의적 사고를 시도하여 문제를 해결하기 위한 혁신적이고 틀에 얽매이지 않는 아이디어를 생성하세요. 비전통적인 솔루션을 탐색하고, 전통적인 경계를 넘어 사고하며, 상상력과 독창성을 장려하세요.",
    # "12. 문제를 해결하기 위해 다른 사람들의 의견과 협력을 구하세요. 팀워크, 개방적 의사소통 및 그룹의 다양한 관점과 전문성을 활용하여 효과적인 솔루션을 찾는 것을 강조합니다.",
    "13. 시스템 사고 사용: 문제를 더 큰 시스템의 일부로 간주하고 다양한 요소의 상호 연결성을 이해합니다. 문제에 영향을 미치는 근본 원인, 피드백 루프 및 상호 의존성을 식별하고 시스템 전체를 다루는 전체적인 솔루션을 개발하는 데 중점을 둡니다.",
    "14. 위험 분석 사용: 문제에 대한 다양한 솔루션이나 접근 방식과 관련된 잠재적 위험, 불확실성 및 절충안을 평가합니다. 잠재적 결과와 성공 또는 실패 가능성을 평가하고 위험과 이점의 균형 잡힌 분석을 기반으로 정보에 입각한 결정을 내리는 것을 강조합니다.",
    # "15. 반성적 사고 사용: 문제에서 한 걸음 물러나 내성과 자기 성찰의 시간을 가지세요. 문제 해결에 영향을 미칠 수 있는 개인적 편견, 가정 및 정신 모델을 검토하고 과거 경험에서 배워 미래 접근 방식을 개선하는 데 개방적입니다.",
    "16. 해결해야 할 핵심 이슈나 문제는 무엇입니까?",
    "17. 문제에 기여하는 근본 원인이나 요인은 무엇입니까?",
    "18. 이전에 시도한 잠재적 솔루션이나 전략이 있습니까? 그렇다면 결과와 교훈은 무엇이었습니까?",
    "19. 이 문제를 해결하는 데 발생할 수 있는 잠재적 장애물이나 도전 과제는 무엇입니까?",
    "20. 문제에 대한 통찰력을 제공할 수 있는 관련 데이터나 정보가 있습니까? 그렇다면 사용 가능한 데이터 소스는 무엇이며 어떻게 분석할 수 있습니까?",
    "21. 문제의 직접적인 영향을 받는 이해관계자나 개인이 있습니까? 그들의 관점과 요구는 무엇입니까?",
    "22. 문제를 효과적으로 해결하는 데 어떤 자원(재정적, 인적, 기술적 등)이 필요합니까?",
    "23. 문제 해결의 진행 상황이나 성공을 어떻게 측정하거나 평가할 수 있습니까?",
    "24. 어떤 지표나 메트릭을 사용할 수 있습니까?",
    "25. 문제가 특정 전문 지식이나 기술 세트를 필요로 하는 기술적 또는 실용적인 것입니까? 아니면 개념적 또는 이론적 문제입니까?",
    "26. 문제에 제한된 자원, 인프라 또는 공간과 같은 물리적 제약이 포함됩니까?",
    "27. 문제가 사회적, 문화적 또는 심리적 이슈와 같은 인간 행동과 관련이 있습니까?",
    "28. 문제에 불확실성 하에서 또는 경쟁하는 목표와 함께 선택이 이루어져야 하는 의사 결정이나 계획이 포함됩니까?",
    "29. 문제가 데이터 분석, 모델링 또는 최적화 기술을 필요로 하는 분석적 문제입니까?",
    "30. 문제가 창의적인 솔루션과 혁신을 필요로 하는 디자인 챌린지입니까?",
    "31. 문제가 개별 인스턴스가 아닌 시스템적 또는 구조적 이슈를 다루어야 합니까?",
    "32. 문제가 즉각적인 주의와 조치를 필요로 하는 시간에 민감하거나 긴급한 것입니까?",
    "33. 이러한 종류의 문제 사양에 대해 일반적으로 어떤 종류의 솔루션이 생성됩니까?",
    "34. 문제 사양과 현재 최선의 솔루션을 고려할 때 다른 가능한 솔루션에 대해 추측해 보세요."
    "35. 현재 최선의 솔루션이 완전히 잘못되었다고 상상해 봅시다. 문제 사양에 대해 생각하는 다른 방법이 있습니까?"
    "36. 이러한 종류의 문제 사양에 대해 알고 있는 것을 고려할 때 이 현재 최선의 솔루션을 수정하는 가장 좋은 방법은 무엇입니까?"
    "37. 현재 최선의 솔루션을 무시하고 문제에 대한 완전히 새로운 솔루션을 만드세요."
    # "38. 단계별로 생각해 봅시다."
    "39. 단계별 계획을 세우고 좋은 표기법과 설명으로 구현해 봅시다.",
]


task_example = "Lisa has 10 apples. She gives 3 apples to her friend and then buys 5 more apples from the store. How many apples does Lisa have now?"

task_example = """This SVG path element <path d="M 55.57,80.69 L 57.38,65.80 M 57.38,65.80 L 48.90,57.46 M 48.90,57.46 L
45.58,47.78 M 45.58,47.78 L 53.25,36.07 L 66.29,48.90 L 78.69,61.09 L 55.57,80.69"/> draws a:
(A) circle (B) heptagon (C) hexagon (D) kite (E) line (F) octagon (G) pentagon(H) rectangle (I) sector (J) triangle"""

reasoning_modules_str = "\n".join(reasoning_modules)

for s in app.stream(
    {"task_description": task_example, "reasoning_modules": reasoning_modules_str}
):
    print(s)